In [1]:
import sys
sys.path.append('/home/mz1482/project/vt-bayesian-opt-bopt_debug/')
from models.ccmodel import CCModel
from models.rsmodel import RSModel
from models.bomodel import BOModel
from models.confi import *
from models.utilfuncs import *
from graph import *
import pandas as pd
from tqdm import tqdm
import random

# Ignoring warnings
import warnings
warnings.filterwarnings('ignore')

In [2]:
data_path = "/home/mz1482/project/vt-bayesian-opt-bopt_debug/data/simu_data_4000/"
# Reading in the ECGs and labels
aucs = pd.read_csv(data_path+"Heart1/Heart1_AUCS.csv", header=None).to_numpy()
ecgs = pd.read_csv(data_path+"Heart1/Heart1_SimuData_4000.csv", header=None).to_numpy()
labels = pd.read_csv(data_path+"Heart1/Coord1_4000.csv", header=None).to_numpy() / 1000
uvc = pd.read_csv(data_path+"UVC/Coord1_UVC_4000.csv", header=None).to_numpy()

In [3]:
total_cases = 0
alle, all_points = [], []
cc_euclids = [[] for _ in range(NUM_STEPS + 21)]                    # Random init CC arrays
cc_successes, cc_avg_sites = [], []
cc_drop = []
# Set mm threshold for finding nearest
mm_thres = 200000

In [4]:
def get_random_dataset(data, coords):
    """
    Handles getting the random dataset for the randomly initialized models
    :param target_coord: target coordinate
    :param data: full dataset for a patient
    :param coords: full labels for a patient
    :return: x, y of size 4
    """
    indices = np.random.choice(range(0, data.shape[0]), NUM_POINTS_START, replace=False)
    return data[indices], coords[indices]

In [5]:
def model_run(model, x, y, train, labels, target, target_coord, target_raw, successes, avg_sites, all_euclids):
    """
    Handles running a single runthrough of a given model on a target site
    Train and labels are the initialized sets to start with (either random or by segment prediction)
    """
    # Run the training loop for the model
    euclids, preds, sites, success, nsites = model.run(x, y, train, labels, target, target_coord, target_raw)
    sites = np.asarray(sites)

    # Add to arrays if successful and add the per steps error
    if success:
        successes.append(euclids[-1])
        avg_sites.append(nsites)
    for j in range(len(euclids)):
        all_euclids[j].append(euclids[j])
        # if j != 0:
        #     drop.append(euclids[j] - euclids[j - 1])
    return successes, avg_sites, all_euclids, sites

In [6]:
cc_model = CCModel(leads=LEADS, steps=10, svr_c=SVR_C, cc=CC_THRES, cc_succ=CC_SUCC,
                   mm=mm_thres, samp_raw=ecgs, samp_coords=labels)
rs_model = RSModel(steps=NUM_STEPS, svr_c=SVR_C, samp_raw=ecgs, samp_coords=labels, cc_succ=CC_SUCC)

In [7]:
idx = np.random.randint(0,1999,1)[0]
target = aucs[idx]
target_coord = labels[idx]
target_raw = ecgs[idx]
target_uvc = uvc[idx]

# Drop the target from the training set
if idx == 0:
    x, y, raw = aucs[idx + 1:, :], labels[idx + 1:], ecgs[idx + 1:, :]
else:
    x = np.concatenate((aucs[:idx, :], aucs[idx + 1:, :]))
    y = np.concatenate((labels[:idx], labels[idx + 1:]))
    raw = np.concatenate((ecgs[:idx], ecgs[idx + 1:]))
random_x, random_y = get_random_dataset(x, y)

cc_euclids, cc_preds, cc_sites, success, num_sites = cc_model.run(x, y, random_x, random_y,
                                                         target, target_coord, target_raw)

In [8]:
lab = uvc[:,0:3]
lab.shape

(4000, 3)

In [9]:
cc_sites

[array([-14.5733, -76.4582, -34.747 ]),
 array([ -8.5367, -64.5615, -39.0669]),
 array([ -6.45628, -63.0949 , -34.9386 ]),
 array([-16.3808, -58.4224, -39.4019]),
 array([-12.8766, -60.0958, -45.1094]),
 array([-15.342 , -55.0044, -34.8803]),
 array([ -8.27326, -63.7605 , -36.9956 ]),
 array([-11.1218, -66.1183, -45.3213]),
 array([ -9.32303, -66.0308 , -41.7344 ]),
 array([-20.9479, -71.4986, -33.4704])]

In [10]:
target

array([-0.0129531 , -0.69461085, -0.68165774,  0.35378197,  0.33435232,
       -0.68813429,  0.17849338,  0.09662747, -0.28807973, -1.09335126,
       -1.48810547, -1.00319581])

In [11]:
ecgs.shape

(4000, 7212)

In [12]:
target

array([-0.0129531 , -0.69461085, -0.68165774,  0.35378197,  0.33435232,
       -0.68813429,  0.17849338,  0.09662747, -0.28807973, -1.09335126,
       -1.48810547, -1.00319581])

In [13]:
random_y.shape

(10, 3)

In [14]:
pred_raw = np.reshape(ecgs, [12, -1])

In [15]:
pred_raw.shape

(12, 2404000)

In [16]:
target_raw.shape

(7212,)

In [17]:
target_raw = np.reshape(target_raw, [12, -1])

In [18]:
target_raw.shape

(12, 601)

In [25]:
exp = np.random.randint(0,3999,5)

In [27]:
idx = exp[1]

In [28]:
idx

2744